In [ ]:
import os
from climada.hazard import TCTracks
import numpy as np

# os.chdir('C:/Users/EB735DB/climada_workspace/')
import dill
# filepath = 'Historical typhoon_China_51tracks.pkl'
# dill.load_session(filepath)
import cartopy.crs as ccrs
import shapely
import time
from cartopy.io import shapereader
from climada.hazard import Centroids, TropCyclone

filepath_output = 'eai_51tracks_China.pkl'

csv_output = 'impact_China_51tracks.csv'

v_half_wp1 = 80.2
from datetime import datetime

###### step 1

In [ ]:
## Load cyclones from cma provider
sel_ibtracs = TCTracks.from_ibtracs_netcdf(provider = 'cma',year_range=(1960,2022), correct_pres=False)
print('Number of tracks:', sel_ibtracs.size)
storm_num = sel_ibtracs.size
sel_ibtracs_raw = sel_ibtracs

In [ ]:
## Simulate syntheic tracks
from pathos.pools import ProcessPool as Pool
pool = Pool() # start a pathos pool
sel_ibtracs.equal_timestep(0.5,pool=pool)  # Interpolation to make the track smooth and to allow applying calc_perturbed_trajectories
# Add randomly generated tracks using the calc_perturbed_trajectories method (1 per historical track)
sel_ibtracs.calc_perturbed_trajectories(nb_synth_tracks=50,pool=pool)
print('num tracks hist+syn:', sel_ibtracs.size)

###### step 2

In [ ]:
shp_file = 'C:/Users/CZ551FU/climada_workspace/Datasets/China_Aliyun/China.shp'
"""initiate LitPop exposures for a geographical box around China:"""
reader = shapereader.Reader(shp_file) # read shpfile of China
# LOGGER.info("Computing earth's land geometry ...")
geom = list(reader.geometries())
geom_new = []
for i in range(len(geom)):
    geom_new.append(geom[i].buffer(0.01))
geom_new = shapely.ops.unary_union(geom_new)

from climada.entity import LitPop
total_value=1
# exp_lp = LitPop.from_countries(countries=['HK'], res_arcsec=30, fin_mode='income_group')
exp_lp = LitPop.from_shape(geom_new, total_value, res_arcsec=360) # rougly 1km
exp_lp.check()
exp_lp.gdf.value = 1
exp_lp.gdf

In [ ]:
#### only slics longitude
z = 0 + 1
min_lat, max_lat, min_lon, max_lon = exp_lp.gdf.latitude.min() - 0.1,exp_lp.gdf.latitude.max() + 0.1,exp_lp.gdf.longitude.min() - 0.1,exp_lp.gdf.longitude.max() + 0.1
d = {}
for y in range(1,63):
    d["min_lat{0}".format(z)] = min_lat 
    d["max_lat{0}".format(z)] = max_lat
    if y == 62:
        d["min_lat{0}".format(z)] = min_lat 
        d["max_lat{0}".format(z)] = max_lat 
        d["min_lon{0}".format(z)] = min_lon + (y-1) * 1
        d["max_lon{0}".format(z)] = max_lon        
    else:
        d["min_lon{0}".format(z)] = min_lon + (y-1) * 1
        d["max_lon{0}".format(z)] = min_lon + (y) * 1
    z = z + 1

In [ ]:
# save cent
from climada.hazard import Centroids, TropCyclone
cent_dict = {}
for i in range(1,int(len(d)/4)):
    cent_dict['cent{}'.format(i)] = Centroids.from_pnt_bounds((d['min_lon{}'.format(i)], d['min_lat{}'.format(i)], d['max_lon{}'.format(i)], d['max_lat{}'.format(i)]), res=0.1) #~11.1km

In [ ]:
# generate the cyclone tracks
tc_dict = {}
for i in range(1,int(len(d)/4)):
    tc_dict['tc_{}'.format(i)] = TropCyclone.from_tracks(sel_ibtracs, centroids=cent_dict['cent{}'.format(i)],pool=pool)
    print('finished tc{}'.format(i) + str(datetime.now()))

In [ ]:
def append_new(lst, *others):

    # pylint: disable=no-member, protected-access
    if len(others) == 0:
        return
    haz_list = [lst] + list(others)
    haz_list_nonempty = [haz for haz in haz_list if haz.size > 0]

#     for haz in haz_list:
#         haz._check_events()

    # check type, unit, and attribute consistency among hazards
    haz_types = {haz.tag.haz_type for haz in haz_list if haz.tag.haz_type != ''}
    if len(haz_types) > 1:
        raise ValueError(f"The given hazards are of different types: {haz_types}. "
                         "The hazards are incompatible and cannot be concatenated.")
    lst.tag.haz_type = haz_types.pop()

    haz_classes = {type(haz) for haz in haz_list}
    if len(haz_classes) > 1:
        raise TypeError(f"The given hazards are of different classes: {haz_classes}. "
                        "The hazards are incompatible and cannot be concatenated.")

    units = {haz.units for haz in haz_list if haz.units != ''}
    if len(units) > 1:
        raise ValueError(f"The given hazards use different units: {units}. "
                         "The hazards are incompatible and cannot be concatenated.")
    if len(units) == 0:
        units = {''}
    lst.units = units.pop()

    attributes = sorted(set.union(*[set(vars(haz).keys()) for haz in haz_list]))
    for attr_name in attributes:
        if not all(hasattr(haz, attr_name) for haz in haz_list_nonempty):
            raise ValueError(f"Attribute {attr_name} is not shared by all hazards. "
                             "The hazards are incompatible and cannot be concatenated.")

    # append all tags (to keep track of input files and descriptions)
    for haz in haz_list:
        if haz.tag is not lst.tag:
            lst.tag.append(haz.tag)

    # map individual centroids objects to union
    centroids = Centroids.union(*[haz.centroids for haz in haz_list])
    hazcent_in_cent_idx_list = [
        u_coord.assign_coordinates(haz.centroids.coord, centroids.coord, threshold=0)
        for haz in haz_list_nonempty
    ]

    # concatenate array and list attributes of non-empty hazards
    for attr_name in attributes:
        attr_val_list = [getattr(haz, attr_name) for haz in haz_list_nonempty]
        if isinstance(attr_val_list[0], sparse.csr.csr_matrix):
            # map sparse matrix onto centroids
            setattr(lst, attr_name, sparse.vstack([
                sparse.csr_matrix(
                    (matrix.data, cent_idx[matrix.indices], matrix.indptr),
                    shape=(matrix.shape[0], centroids.size)
                )
                for matrix, cent_idx in zip(attr_val_list, hazcent_in_cent_idx_list)
            ], format='csr'))
        elif isinstance(attr_val_list[0], np.ndarray) and attr_val_list[0].ndim == 1:
            setattr(lst, attr_name, np.hstack(attr_val_list))
        elif isinstance(attr_val_list[0], list):
            setattr(lst, attr_name, sum(attr_val_list, []))

    lst.centroids = centroids
    lst.sanitize_event_ids()


In [ ]:
import climada.util.coordinates as u_coord
from scipy import sparse
for i in range(1,int(len(d)/4)):
    print(i)
    append_new(tc_dict['tc_1'],tc_dict['tc_{}'.format(i+1)])

In [ ]:
tc_dict['tc_1'].plot_intensity(0)

In [ ]:
# tc.check()
for haz in [tc_dict['tc_1']]:
    print(len(haz.event_id))
    print(len(haz._events_set()))

In [ ]:
"""Set up impact function"""
from climada.entity import ImpactFuncSet, ImpfTropCyclone
# impact function TC
impf_tc = ImpfTropCyclone.from_emanuel_usa(v_half=v_half_wp1) # WP1

# add the impact function to an Impact function set
impf_set = ImpactFuncSet()
impf_set.append(impf_tc)
impf_set.check()
 
# Get the hazard type and hazard id
[haz_type] = impf_set.get_hazard_types()
[haz_id] = impf_set.get_ids()[haz_type]
#print(f"hazard type: {haz_type}, hazard id: {haz_id}")

In [ ]:
from climada.engine import Impact
imp = Impact()
imp.calc(exp_lp, impf_set, tc_dict['tc_1'], save_mat=False)

In [ ]:
imp.unit = "unitless"

In [ ]:
imp.plot_scatter_eai_exposure(ignore_zero=False, buffer=0.03);

In [ ]:
import pandas as pd
data={
    'value': imp.eai_exp,
    'latitude': imp.coord_exp[:, 0],
    'longitude': imp.coord_exp[:, 1],
}
imp_data = pd.DataFrame(data)
#imp_data

In [ ]:
## The centroid has max windspeed
max_cent = imp_data.iloc[imp_data[['value']].idxmax()[0]]
max_cent_lat = max_cent.latitude
max_cent_lon = max_cent.longitude
max_cent_eai = max_cent.value

In [ ]:
print("Eai_mean",np.mean(imp.eai_exp),
      "Eai_max",max_cent_eai,"where is at", max_cent_lat, max_cent_lon)

In [ ]:
csv_output = 'impact_China_51tracks.csv'
imp_data.to_csv(csv_output,index=False)

In [ ]:
dill.dump_session(filepath_output)